# The Battle for Neighborhood (week 01)

TABLE OF CONTENT:

1. Introduction: Discuss the buisness problem and get familiar to the project.
2. Data Selection: Description of valuable data resources to be used to complete the project.
3. Methodology: Application of data cleaning, sorting, analysis, applying machine learning algorithms, data taining & testing in order to establish the purpose to reach the target.
4. Result: Get familiar to the results or output
5. Discussion: Explaination of the results and discuss the outcome.
6. Conclusion: Finalize report


In [1]:
# A description of the problem and a discussion of the background. (15 marks)

## 1. Introduction: 
This project is regarding the comparison of neighborhoods of two cities namely "Manhattan NY" and "Toronto" in choosing the best place to live provided all the necessary ammenities around.

The basic idea behind this comparison is that I have got new job offers as a data scientist in "Manhattan NY" and another in "Toronto". There are some basic evaluation criteria which I would like to check by means of this project in searching for apartment and living standards in the two cities. Follwing are the basic queries to be addressed:

1. Two bedroom or three bedroom apartment.
2. Metro station must be 1 mile from the place of living.
3. Montly rent not more than $1000.
4. Top ammenities must be included in both places.
5. Resturants, groceries stores, wine shop are desirable nearby.

### Problems:
The difficult part is to locate a suitable apartment for rent in both places "Manhattan NY" and "Toronto" that fulfil the demands of price, location and ammenities. In the following section 2 I will explain the process of data collection and importance while addressing the issues. 

### Import basic libraries required for this project

In [19]:
import numpy as np 
import time
import pandas as pd 
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

import json 
import requests 
from pandas.io.json import json_normalize 

from geopy.geocoders import Nominatim 
import geopy
import folium 

print('Libraries imported.')

Libraries imported.


In [20]:
# A description of the data and how it will be used to solve the problem. (15 marks)

## 2. Data formulation:

### Following data are required for the project
1. List of Boroughs and neighborhoods of Manhattan and Toronto with their geodata (Latitude and Longitude).
2. List of subways metro stations with their address location.
3. List of apartment for rent in Manhattan and Toronto.
4. Desired additional information about the price, number of beds etc.
5. Venues for each neighborhoods in Manhattan and Toronto.
6. Venues for subway metro stations.

### How to solve problem using above data?
We will use the data in follwing way:
1. Use Foursqure and geopy data to sort the top 10 venues in "Toronto" and "Manhattan" neighborhood and cluster them in group. Then locate the subway stations seperately and layer it over the created cluster group.
2. Again use the same geopy and foursqure data to map the location of rental places. 
3. Filter the data with the average rental price $1000 per month and radius of 1 mile near the subway station. 
4. Convert the address of the rental location to latitude and logitute.



In [22]:
address1 = "Mill Street, Toronto"
geolocator = Nominatim(user_agent = "my-application")
location = geolocator.geocode(address1)
latitude = location.latitude
longitude = location.longitude
#geopy.geocoders.options.default_user_agent = "my-application"
print('Geogrophical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

Geogrophical coordinates of Toronto are 43.6520276, -79.3545845.


In [23]:
toronto_latitude=43.6520276
toronto_longitude=-79.3545845

In [75]:
# @hidden cell
CLIENT_ID = 
CLIENT_SECRET = 
VERSION = '20180605'

In [76]:
LIMIT = 200 # number of venues returned by Foursquare API
radius = 1000 # radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    toronto_latitude, 
    toronto_longitude, 
    radius, 
    LIMIT)
#url    # url is commented 

In [81]:
get_json = requests.get(url).json()
with open('toronto.json', 'w') as json_file:
    json.dump(get_json, json_file)
# Get json file

In [82]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [83]:
venues = get_json['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Souk Tabule,Mediterranean Restaurant,43.653756,-79.354390
1,SOMA chocolatemaker,Chocolate Shop,43.650622,-79.358127
2,Young Centre for the Performing Arts,Performing Arts Venue,43.650825,-79.357593
3,Toronto Cooper Koo Family Cherry St YMCA Centre,Gym / Fitness Center,43.653191,-79.357947
4,Cluny Bistro & Boulangerie,French Restaurant,43.650565,-79.357843
5,The Distillery Historic District,Historic Site,43.650244,-79.359323
6,Underpass Park,Park,43.655764,-79.354806
7,Sukhothai,Thai Restaurant,43.653701,-79.354447
8,Balzac's Coffee,Coffee Shop,43.649797,-79.359142
9,Corktown Common,Park,43.655618,-79.356211


In [84]:
# Toronto Map with latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=3,
        radius=8,
        popup=label,
        color='orange',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg